In [33]:
import torch 
import torch.nn as nn
from prettytable import PrettyTable

In [34]:
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [35]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)  # W^O matrix 

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        # Concatenation
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        # Output of the multi-head self-attention
        return out

In [36]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        # x = self.dropout( LayerNorm(Sublayer(x) + x) )

        x = self.norm1(attention + query)

        print("Value after normalisation. (Before FF)")
        print(x)

        x = self.dropout(x)

        forward = self.feed_forward(x)

        x = self.norm2(forward + x)

        print("Value after normalisation. (After FF)")
        print(x)

        out = self.dropout(x)

        return out

In [37]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)
        self.encoder_layer_counter = 0

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        # The last Encoder output is the same as the final Encoder output
        for layer in self.layers:
            self.encoder_layer_counter = self.encoder_layer_counter + 1
            out = layer(out, out, out, mask)
            print(f"{self.encoder_layer_counter}-th Encoder output is {out}")

        print("<Encoder out shape>\n", out.shape)
        print("<Encoder out>\n", out)    
        print("================================================================")

        return out

In [38]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


In [39]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.decoder_layer_counter = 0 

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            self.decoder_layer_counter = self.decoder_layer_counter + 1
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
            print("Shape of Decoder output", x.shape)
            print(f"{self.decoder_layer_counter}-th Decoder output is {x}")
        
        out = self.fc_out(x)

        print("\n<Output of decoder after applying Linear layer>\n")
        print("Decoder out.shape", out.shape)
        print("Decoder out", out)

        return out


In [40]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out


In [41]:
if __name__ == "__main__":

    print("=============================== main started ===============================")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2], [1, 8, 7, 3, 4, 5, 6, 7]]).to(
        device
    )

    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10

    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
        device
    )

    out = model(x, trg)

    print("\n<Output of decoder after applying Linear layer>\n")
    print(out.shape)
    print(out)

    print("=============================== main done ===============================")

=============================== main started ===============================
cpu
Value after normalisation. (Before FF)
tensor([[[ 0.1560, -0.3566,  1.2330,  ...,  1.6240,  1.0576,  0.8888],
         [-0.7901, -0.4820,  1.3944,  ...,  1.4702, -0.1224,  0.9340],
         [-0.2097, -0.4023, -1.6194,  ..., -0.8245, -0.4463, -2.0399],
         ...,
         [ 0.4298, -0.3077, -0.1539,  ..., -0.3422, -1.0317, -0.3408],
         [ 0.8044,  0.5168,  2.2114,  ..., -0.3697, -0.7543, -0.1361],
         [-1.5110,  0.8067, -1.7293,  ...,  1.4637, -0.4376,  0.0859]],

        [[ 0.1244, -0.3457,  1.2885,  ...,  1.7708,  1.1376,  0.8630],
         [-0.8464, -0.3012, -0.7076,  ..., -0.2684, -0.2277,  2.4080],
         [ 1.0087, -0.6809, -0.2126,  ...,  0.3562, -0.4049, -0.4209],
         ...,
         [ 0.7371, -0.5263,  1.9929,  ...,  0.4190, -0.3427,  0.1182],
         [ 0.3449,  0.0600,  0.3378,  ..., -1.7634, -1.4093, -0.7624],
         [ 0.6100, -0.0232,  0.6457,  ...,  2.0002, -0.7560,  1.3350]

In [42]:
count_parameters(model)

+-------------------------------------------------------------+------------+
|                           Modules                           | Parameters |
+-------------------------------------------------------------+------------+
|                encoder.word_embedding.weight                |    5120    |
|              encoder.position_embedding.weight              |   51200    |
|           encoder.layers.0.attention.values.weight          |    4096    |
|            encoder.layers.0.attention.keys.weight           |    4096    |
|          encoder.layers.0.attention.queries.weight          |    4096    |
|           encoder.layers.0.attention.fc_out.weight          |   262144   |
|            encoder.layers.0.attention.fc_out.bias           |    512     |
|                encoder.layers.0.norm1.weight                |    512     |
|                 encoder.layers.0.norm1.bias                 |    512     |
|                encoder.layers.0.norm2.weight                |    512     |

30294026